In [1]:
import pandas as pd
import os
from typing import List
import glob

In [2]:
test_folder = "./../test"
file_list = glob.glob(os.path.join(test_folder, "*.csv"))
print("Files found:", file_list)

Files found: ['./../test\\01. CG AIS.csv', './../test\\02. CG AIS.csv', './../test\\03. CG AIS.csv', './../test\\04. CG AIS.csv', './../test\\05. CG AIS.csv', './../test\\06. CG AIS.csv']


In [3]:
file_paths = file_list
output_path = r"./../test/combined_output.csv"

In [4]:
            
for i, file_path in enumerate(file_list):
    if not os.path.exists(file_path):
        print(f"Warning: File {file_path} does not exist, skipping...")
        continue

In [5]:
headers =[
                "Security Name (Security Code)",
                "Date of Sale/Transfer",
                "Asset Type",
                "Quantity",
                "Sales Consideration - Reported by Source",
                "Cost of Acquisition",
                "Status"
                
            ]

In [6]:

combined_data = []
fno = 1
df = pd.read_csv(file_path,skiprows=1)
df.dtypes
                
                

S.No                                          int64
Information Category                         object
Information Code                             object
Information Description                      object
Information Source                           object
Amount Description                           object
Amount                                       object
AMC Name (Code)                              object
Date of Sale/Transfer                        object
Security Class                               object
Security Name (Security Code)                object
Debit Type                                   object
Credit Type                                  object
Asset Type                                   object
Quantity                                    float64
Sale Price Per unit                          object
Sales Consideration - Reported by Source     object
Sales Consideration - Modified              float64
STT                                           int64
Cost of Acqu

In [7]:
# Set the column names to match the headers from the first file
df = df[headers]
df.head()
# df.dtypes

,Security Name (Security Code),Date of Sale/Transfer,Asset Type,Quantity,Sales Consideration - Reported by Source,Cost of Acquisition,Status
0,quant Liquid Fund_Direct Plan_Growth(INF966L01...,11-NOV-2024,Short term,50.72,"2,056","2,004",Active
1,quant Liquid Fund_Direct Plan_Growth(INF966L01...,11-NOV-2024,Long term,28.84,"1,169","1,000",Active
2,quant Liquid Fund_Direct Plan_Growth(INF966L01...,11-NOV-2024,Short term,28.20,"1,143","1,000",Active
3,quant Liquid Fund_Direct Plan_Growth(INF966L01...,11-NOV-2024,Long term,22.26,902,700,Active
4,quant Liquid Fund_Direct Plan_Growth(INF966L01...,11-NOV-2024,Long term,3.18,129,100,Active


In [8]:

                
# Convert columns to numeric, removing commas and handling errors
df['Cost of Acquisition'] = pd.to_numeric(df['Cost of Acquisition'].replace({r',': ''}, regex=True))
df['Sales Consideration - Reported by Source'] = pd.to_numeric(df['Sales Consideration - Reported by Source'].replace({r',': ''}, regex=True), errors='coerce')

# Add source file column to track origin
df['source_file'] = os.path.basename(file_path)
df['Sale - Cost'] = df['Sales Consideration - Reported by Source'] - df['Cost of Acquisition']

# Convert the 'Date of Sale/Transfer' column to datetime if not already
df['Date of Sale/Transfer'] = pd.to_datetime(df['Date of Sale/Transfer'],format="%d-%b-%Y", errors='coerce', dayfirst=True)
df.dtypes
df.head()

,Security Name (Security Code),Date of Sale/Transfer,Asset Type,Quantity,Sales Consideration - Reported by Source,Cost of Acquisition,Status,source_file,Sale - Cost
0,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Short term,50.72,2056,2004,Active,06. CG AIS.csv,52
1,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Long term,28.84,1169,1000,Active,06. CG AIS.csv,169
2,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Short term,28.20,1143,1000,Active,06. CG AIS.csv,143
3,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Long term,22.26,902,700,Active,06. CG AIS.csv,202
4,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Long term,3.18,129,100,Active,06. CG AIS.csv,29


In [9]:
# df = df[df['Asset Type'] == "Short term"] # No need to filter by Asset Type
df = df[df['Status'] == "Active"]
df['31 July 2024'] = df['Date of Sale/Transfer'].apply(
    lambda x: "Before 31 July 2024" if x < pd.to_datetime("2024-07-31") else "After 31 July 2024"
)

df.head()

,Security Name (Security Code),Date of Sale/Transfer,Asset Type,Quantity,Sales Consideration - Reported by Source,Cost of Acquisition,Status,source_file,Sale - Cost,31 July 2024
0,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Short term,50.72,2056,2004,Active,06. CG AIS.csv,52,After 31 July 2024
1,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Long term,28.84,1169,1000,Active,06. CG AIS.csv,169,After 31 July 2024
2,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Short term,28.20,1143,1000,Active,06. CG AIS.csv,143,After 31 July 2024
3,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Long term,22.26,902,700,Active,06. CG AIS.csv,202,After 31 July 2024
4,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Long term,3.18,129,100,Active,06. CG AIS.csv,29,After 31 July 2024


In [10]:
# Clean up any empty rows
df = df.dropna(how='all')
df.head()         

,Security Name (Security Code),Date of Sale/Transfer,Asset Type,Quantity,Sales Consideration - Reported by Source,Cost of Acquisition,Status,source_file,Sale - Cost,31 July 2024
0,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Short term,50.72,2056,2004,Active,06. CG AIS.csv,52,After 31 July 2024
1,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Long term,28.84,1169,1000,Active,06. CG AIS.csv,169,After 31 July 2024
2,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Short term,28.20,1143,1000,Active,06. CG AIS.csv,143,After 31 July 2024
3,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Long term,22.26,902,700,Active,06. CG AIS.csv,202,After 31 July 2024
4,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Long term,3.18,129,100,Active,06. CG AIS.csv,29,After 31 July 2024


In [11]:
combined_data.append(df)

In [14]:
            
# Combine all dataframes
combined_df = pd.concat(combined_data, ignore_index=True)
combined_df.head()

,Security Name (Security Code),Date of Sale/Transfer,Asset Type,Quantity,Sales Consideration - Reported by Source,Cost of Acquisition,Status,source_file,Sale - Cost,31 July 2024
0,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Short term,50.72,2056,2004,Active,06. CG AIS.csv,52,After 31 July 2024
1,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Long term,28.84,1169,1000,Active,06. CG AIS.csv,169,After 31 July 2024
2,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Short term,28.20,1143,1000,Active,06. CG AIS.csv,143,After 31 July 2024
3,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Long term,22.26,902,700,Active,06. CG AIS.csv,202,After 31 July 2024
4,quant Liquid Fund_Direct Plan_Growth(INF966L01...,2024-11-11,Long term,3.18,129,100,Active,06. CG AIS.csv,29,After 31 July 2024


In [13]:
            
# Save to output path
combined_df.to_csv(output_path, index=False)
            
print(f"Successfully combined {len(file_paths)} files into {output_path}")
print(f"Total data rows: {len(combined_df)}")
print(f"Total columns: {len(combined_df.columns)}")
            

Successfully combined 6 files into ./../test/combined_output.csv
Total data rows: 9
Total columns: 10
